# Jupyter Notebook approach for SCORE interaction


### Necessary packages 

In [1]:
from checkscore.methods import readonly, external, payable
from checkscore.methods import Checkscore
from iconsdk.wallet.wallet import KeyWallet

password = input()
deployer_wallet = KeyWallet.load("keystore_ibriz.json",password)
deployer_wallet.get_address()

SCORE_ADDRESS = ""
Checkscore(SCORE_ADDRESS).fill_methods()

ModuleNotFoundError: No module named 'checkscore'

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [8]:
from iconsdk.exception import JSONRPCException
from iconsdk.libs.in_memory_zip import gen_deploy_data_content
from iconsdk.icon_service import IconService
from iconsdk.providers.http_provider import HTTPProvider
from iconsdk.builder.transaction_builder import CallTransactionBuilder, TransactionBuilder, DeployTransactionBuilder
from iconsdk.builder.call_builder import CallBuilder
from iconsdk.signed_transaction import SignedTransaction
from iconsdk.wallet.wallet import KeyWallet
from iconsdk.utils.convert_type import convert_hex_str_to_int
from repeater import retry
import requests
import json
import pickle
import csv
import os
from pprint import pprint
import getpass

@retry(JSONRPCException, tries=10, delay=1, back_off=2)
def get_tx_result(_tx_hash):
    tx_result = icon_service.get_transaction_result(_tx_hash)
    return tx_result

mainnet= {
    "iconservice":"https://ctz.solidwallet.io",
    "nid" :1
    
}

yeouido={
     "iconservice":"https://bicon.net.solidwallet.io",
     "nid" : 3
    
}
euljiro={
     "iconservice":"https://test-ctz.solidwallet.io",
     "nid" : 2
    
}
pagoda={
    "iconservice":"https://zicon.net.solidwallet.io",
     "nid" :80
    
    
}
custom={
    "iconservice":"http://18.237.205.52:9000/",
     "nid":3
    
    
}


### Required service

In [15]:
env = custom #mainnet,yeouido,euljiro,pagoda,custom
icon_service = IconService(HTTPProvider(env["iconservice"], 3))
NID = env["nid"]



### A list for SCORE addresses 

In [16]:

GOVERNANCE_ADDRESS = "cx0000000000000000000000000000000000000000"


# Tbears
USDb = "cx072d8f95877a53df350e3dc3d8dba2f379037d42"
oUSDb = "cx48dd26868c25cfa744aab9f2fccd76ed40c41454"
LendingPool= "cxbf1a257dcb378b0fd41066fa87abef3f25992832"
LendingPoolCore = "cx162fe5b5d3276e9493de1fd440511f0844ca9a7c"
LendingPoolDataProvider = "cx84490f03cc04b799fe668545340eaeac7cae628c"
PriceOracle = "cx293582520a75648d91a3c454ad9657aa4cbf1f7d"
AddressProvider = "cxad29953f5b8957704e9092632bccc3b1aac62eea"
FeeProvider = "cxf293e454a0c8a28142da48a136458a177d047769"



# # Yeouido Testnet
# USDb = "cx19be1be7b4e7750863edf966faac3f42dec21025"
# oUSDb = "cx4258464e395b3ef3b73a146947eb41393e8c5873"
# LendingPool= "cx35b12ca4e9c672518db3c06d92ed60d50e7d5554"
# LendingPoolCore = "cx327a386a9f8076045679fc132a8baa9e270ca43b"
# LendingPoolDataProvider = "cxa198eaf0a589adf0ab31b523d1f8912f47f6f293"
# PriceOracle = "cx3f6477bf290eb91257b806ece614ab18b1c7be41"
# AddressProvider = "cxed440f1da2cf3087ac5ef12217a13f6321de0375"
# FeeProvider = "cxede316aa65b7df13633599e67490daf1fd32b8a5"


### Importing necessary wallets

If you want to use your pre-exisiting wallets, execute the cell below:-

In [11]:
password = getpass.getpass()

deployer_wallet = KeyWallet.load("keystore_2",password)
deployer_wallet.get_address()


········


'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce'

### OR creating new wallets 

If you want to use new wallets, execute the cell below:-

In [ ]:
deployer_wallet = KeyWallet.create()
print(deployer_wallet.get_address())
caller_wallet = KeyWallet.create()
print(caller_wallet.get_address())

### Loading ICX
Go to [ibriz faucet](https://icon-faucet.ibriz.ai/) to load ICX into your wallets.

### Deploying contract

In [47]:
# params = {'_initialSupply':500000000, '_decimals': 18}
deploy_transaction = DeployTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(oUSDb)\
    .nid(3)\
    .nonce(100)\
    .content_type("application/zip")\
    .content(gen_deploy_data_content('oToken'))\
    .build()

estimate_step = icon_service.estimate_step(deploy_transaction)
step_limit = estimate_step + 100000
signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)

tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

{'txHash': '0x5d33961060b3244523bdbb5b4acad51e6eeb8c1c3317812bc7e8c3fb22ab37a6',
 'blockHeight': 277804,
 'blockHash': '0xddb3b2e4b410382823efdf25d5a4a9ba71995c0b14706c5281924c09368a54bc',
 'txIndex': 1,
 'to': 'cx48dd26868c25cfa744aab9f2fccd76ed40c41454',
 'scoreAddress': 'cx48dd26868c25cfa744aab9f2fccd76ed40c41454',
 'stepUsed': 1743172000,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 1743172000,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0

### Updating contract

In [39]:
deploy_transaction = DeployTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(LendingPool)\
    .nid(NID)\
    .nonce(1000)\
    .content_type("application/zip")\
    .content(gen_deploy_data_content('lendingPool'))\
    .build()

estimate_step = icon_service.estimate_step(deploy_transaction)
step_limit = estimate_step + 100000
signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)

tx_hash = icon_service.send_transaction(signed_transaction)


get_tx_result(tx_hash)

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


{'txHash': '0x739fd3d28d99e72640a1e5c729a60794f80693a6270f9069a6ebc0f71e0f75ad',
 'blockHeight': 409212,
 'blockHash': '0x3d8b13f41e7a0156e5414f58d583440f25b599e1d50d3fd0ba17f92c8fa4cf4c',
 'txIndex': 1,
 'to': 'cxbf1a257dcb378b0fd41066fa87abef3f25992832',
 'scoreAddress': 'cxbf1a257dcb378b0fd41066fa87abef3f25992832',
 'stepUsed': 1693407200,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 1693407200,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0

## Adding new reserve in LendingPoolCore



In [ ]:
params ={"_reserve": {"reserveAddress":"cx072d8f95877a53df350e3dc3d8dba2f379037d42","oTokenAddress":"cx48dd26868c25cfa744aab9f2fccd76ed40c41454","totalBorrows":"0","lastUpdateTimestamp": "0","liquidityRate":"0","borrowRate":"0","liquidityCumulativeIndex":f"1{'0'*18}","borrowCumulativeIndex":f"1{'0'*18}","baseLTVasCollateral":"60","liquidationThreshold":"65","liquidationBonus":"10","decimals":"18","borrowingEnabled": "1","usageAsCollateralEnabled":"1","isFreezed":"0","isActive":"1"} }
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to("cx162fe5b5d3276e9493de1fd440511f0844ca9a7c") \
    .nid(NID) \
    .nonce(100) \
    .method("addReserveData")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

## Setting reserve constants in LendingPoolCore

In [ ]:
params ={"_constants": [{"reserve":"cx072d8f95877a53df350e3dc3d8dba2f379037d42","optimalUtilizationRate":f"6{'0'*17}","baseBorrowRate":"0","slopeRate1":f"7{'0'*16}","slopeRate2":f"3{'0'*18}"} ]}
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to("cx162fe5b5d3276e9493de1fd440511f0844ca9a7c") \
    .nid(NID) \
    .nonce(100) \
    .method("setReserveConstants")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

## Setting interfaces in LendingPool

In [42]:
params ={"_address": LendingPool }
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(oUSDb) \
    .nid(NID) \
    .nonce(100) \
    .method("setLendingPoolAddress")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

{'version': '0x3', 'from': 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce', 'to': 'cx48dd26868c25cfa744aab9f2fccd76ed40c41454', 'stepLimit': '0x3c460', 'timestamp': '0x5b347ebaa9004', 'nid': '0x3', 'nonce': '0x64', 'dataType': 'call', 'data': {'method': 'setLendingPoolAddress', 'params': {'_address': 'cxbf1a257dcb378b0fd41066fa87abef3f25992832'}}, 'signature': '6qs4PAjl7bIzL2CxbtqhyZCkJdEfsF9wa6UZ1RF3kyEsGHybc1cLI4sG0nTkMvHhx6hlwkUhrw0scAZCuKTp+wA='}


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


{'txHash': '0xefefd9c2ee60b913298a043bf5e9a7654baa50861f8adb19c71e11d1849c5e65',
 'blockHeight': 230186,
 'blockHash': '0xb90b4dc6ee9931d2b47eebfa93e6504f1d5d186f19ff2932a6b6d0835f658b01',
 'txIndex': 1,
 'to': 'cx48dd26868c25cfa744aab9f2fccd76ed40c41454',
 'stepUsed': 146880,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 146880,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x

In [12]:
params ={"_address": "cx072d8f95877a53df350e3dc3d8dba2f379037d42" }
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to("cxbf1a257dcb378b0fd41066fa87abef3f25992832") \
    .nid(NID) \
    .nonce(100) \
    .method("setUSDbAddress")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

{'version': '0x3', 'from': 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce', 'to': 'cxbf1a257dcb378b0fd41066fa87abef3f25992832', 'stepLimit': '0x3d298', 'timestamp': '0x5b309044d16af', 'nid': '0x3', 'nonce': '0x64', 'dataType': 'call', 'data': {'method': 'setUSDbAddress', 'params': {'_address': 'cx072d8f95877a53df350e3dc3d8dba2f379037d42'}}, 'signature': 'QudO9QJkDPtvc5UFbQIgBvjDMc/Vj7RJ5zlKcljU6PU9qKdNNaXZIVLeuAWsCCoNGLaFg88mm+pt3L7QhVeDFgE='}


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


{'txHash': '0x6d25b73a67de8cc1c15422d1bd91b24d987fd7049fe1c2ff095231082048ef2b',
 'blockHeight': 95270,
 'blockHash': '0xa6c15d3ae47aeac3a6d0159aebb685bb1cadad28aee31bf411631542e1f621db',
 'txIndex': 1,
 'to': 'cxbf1a257dcb378b0fd41066fa87abef3f25992832',
 'stepUsed': 150520,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 150520,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0

## Setting interfaces in LendingPoolDataProvider

In [ ]:
params ={"_address": "cx162fe5b5d3276e9493de1fd440511f0844ca9a7c" }
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to("cx84490f03cc04b799fe668545340eaeac7cae628c") \
    .nid(NID) \
    .nonce(100) \
    .method("setLendingPoolCoreAddress")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

In [37]:
params ={"_address": FeeProvider }
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(LendingPool) \
    .nid(NID) \
    .nonce(100) \
    .method("setFeeProvider")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)



# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

{'txHash': '0x771ca8c13fb27e4a474b0e4f8a6e0e5548c394621de98520af71da47437b9045',
 'blockHeight': 230111,
 'blockHash': '0x9b0fd2a6bb8af04cc7db2ec3b837a456de6749b919b69a66bb74746b14684574',
 'txIndex': 1,
 'to': 'cxbf1a257dcb378b0fd41066fa87abef3f25992832',
 'stepUsed': 150520,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 150520,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x

## Setting Price in Oracle

In [ ]:
params ={"_address": "cx162fe5b5d3276e9493de1fd440511f0844ca9a7c" }
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to("cx84490f03cc04b799fe668545340eaeac7cae628c") \
    .nid(NID) \
    .nonce(100) \
    .method("setLendingPoolCoreAddress")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

In [ ]:
params ={"_reserveAddress": "cx072d8f95877a53df350e3dc3d8dba2f379037d42","_liquidityRate":10**18 }
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to("cx162fe5b5d3276e9493de1fd440511f0844ca9a7c") \
    .nid(NID) \
    .nonce(100) \
    .method("updateLiquidityRate")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

## Setting Price in Oracle

In [34]:
params ={"_base": "USDb", "_quote":"USD","_rate": 5 * 10**17 }
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to("cx293582520a75648d91a3c454ad9657aa4cbf1f7d") \
    .nid(NID) \
    .nonce(100) \
    .method("set_reference_data")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, deployer_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

{'version': '0x3', 'from': 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce', 'to': 'cx293582520a75648d91a3c454ad9657aa4cbf1f7d', 'stepLimit': '0x3bf88', 'timestamp': '0x5b359d9761695', 'nid': '0x3', 'nonce': '0x64', 'dataType': 'call', 'data': {'method': 'set_reference_data', 'params': {'_base': 'USDb', '_quote': 'USD', '_rate': '0x6f05b59d3b20000'}}, 'signature': 'xoc94hdu+Sc6ZdL31oIjltkGCRJFGbg9ogT6WSoB/DsDQuS4iQtvGzGVN9t5sddKk6VtKBV38v4KGaTRSXVb6AA='}
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


{'txHash': '0x88f684bc4419ee00c4d3bfbec318f38a822764693bbadba8c5fc38a3b74bf093',
 'blockHeight': 268640,
 'blockHash': '0x4c2025fa7a09f4b9687b0550bdefd2b7b523c63df44989e0332e19512232ea73',
 'txIndex': 1,
 'to': 'cx293582520a75648d91a3c454ad9657aa4cbf1f7d',
 'stepUsed': 145640,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 145640,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x

## Withdraw USDb

In [21]:
params = {"_amount": 100000000000000000000}
transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(oUSDb)\
    .value(0)\
    .step_limit(10000000)\
    .nid(3)\
    .nonce(100)\
    .method("redeem")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, deployer_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

{'txHash': '0x92bf19e380edba727dc293b05b1c8fb8722325b6f0c1edf718e5e8fc5838dad9',
 'blockHeight': 408422,
 'blockHash': '0x60bf7da093c2daf566e02669fbf6610be1eb3049926481c07112eaaf34d66a5e',
 'txIndex': 1,
 'to': 'cx48dd26868c25cfa744aab9f2fccd76ed40c41454',
 'stepUsed': 1114480,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 1114480,
 'eventLogs': [{'scoreAddress': 'cx48dd26868c25cfa744aab9f2fccd76ed40c41454',
   'indexed': ['Mint(Address,int)',
    'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce'],
   'data': ['0x763ac7b8cf9']},
  {'scoreAddress': 'cx48dd26868c25cfa744aab9f2fccd76ed40c41454',
   'indexed': ['Burn(Address,int)',
    'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce'],
   'data': ['0x56bc75e2d63100000']},
  {'scoreAddress': 'cx162fe5b5d3276e9493de1fd440511f0844ca9a7c',
   'indexed': ['ReserveUpdated(Address,int,int,int,int)',
    'cx072d8f95877a53df350e3dc3d8dba2f379037d42',
    '0x112ade44fc2',
    '0x163ab16f662af'],
   'data': ['0xde0b6b66e555971', '0xde0ba0c9ce15fc5']},
  {

In [40]:

data = "{\"method\": \"repay\", \"params\": {\"amount\": 10000000000000000000}}".encode("utf-8")
params = {"_to": LendingPool, "_value": 10000000000000000000, "_data": data}
transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(USDb)\
    .value(0)\
    .step_limit(10000000)\
    .nid(3)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, deployer_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

{'txHash': '0x6a6484fdfaf726b3c344a88c7ee69e7899199adbfbe5680cb6581a9d5b9ae222',
 'blockHeight': 409216,
 'blockHash': '0x3b7800710eb22c4b255e3a541877fae1ba9f633cc41ea1aeb430fa660e411588',
 'txIndex': 1,
 'to': 'cx072d8f95877a53df350e3dc3d8dba2f379037d42',
 'stepUsed': 917100,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 917100,
 'eventLogs': [{'scoreAddress': 'cx162fe5b5d3276e9493de1fd440511f0844ca9a7c',
   'indexed': ['ReserveUpdated(Address,int,int,int,int)',
    'cx072d8f95877a53df350e3dc3d8dba2f379037d42',
    '0xca737158c5',
    '0x131588f9de4f5'],
   'data': ['0xde0b6b671e0f407', '0xde0ba113418579e']},
  {'scoreAddress': 'cx072d8f95877a53df350e3dc3d8dba2f379037d42',
   'indexed': ['Transfer(Address,Address,int,bytes)',
    'cxbf1a257dcb378b0fd41066fa87abef3f25992832',
    'cx162fe5b5d3276e9493de1fd440511f0844ca9a7c',
    '0xdfb5f45b0d9cc27'],
   'data': ['0x4e6f6e65']},
  {'scoreAddress': 'cxbf1a257dcb378b0fd41066fa87abef3f25992832',
   'indexed': ['Repay(Address,Address,in

### ICX Transfer

In [8]:
transaction = TransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to("hx177b10efe3018961a405cc7c20ee811c552831a5")\
    .nid(3) \
    .nonce(100) \
    .value(10000*10**18)\
    .build()

estimate_step = icon_service.estimate_step(transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

{'version': '0x3', 'from': 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce', 'to': 'hx177b10efe3018961a405cc7c20ee811c552831a5', 'stepLimit': '0x30d40', 'timestamp': '0x5b352f9520905', 'nid': '0x3', 'value': '0x10f0cf064dd59200000', 'nonce': '0x64', 'signature': 'k2uHKccWbioV8N5TkAXuffsOprQ90Jb5nMD9M2u3PiBE1WO3h2c0zzDsmNR3/8nq2Ypz20vvh2bVFm2L1kQhFAE='}
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


{'txHash': '0x8f67dc35d0b967bdac94eb8431b7956c0f21d72de631f6e5538cd01612c0379c',
 'blockHeight': 253893,
 'blockHash': '0xde3f86e1c017936f5a60e9ddf7381590423c9c69a440e1420fe42036acea9a9b',
 'txIndex': 1,
 'to': 'hx177b10efe3018961a405cc7c20ee811c552831a5',
 'stepUsed': 100000,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 100000,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x

In [10]:
# hx91bf040426f226b3bfcd2f0b5967bbb0320525ce
balance = icon_service.get_balance("hx91bf040426f226b3bfcd2f0b5967bbb0320525ce")
print(balance/10**18)

78735532.9413376


In [ ]:

_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to("cxad29953f5b8957704e9092632bccc3b1aac62eea")\
    .method("getAllAddress")\
    .build()
response = icon_service.call(_call)
print(response)

In [60]:
params = {'_user': 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce' }

_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(LENDING_POOL_DATA_PROVIDER)\
    .method("getUserAccountData")\
    .params(params)\
    .build()
response = icon_service.call(_call)
print(response)

{'totalLiquidityBalanceUSD': '0x1af8330e6448ec313b', 'totalCollateralBalanceUSD': '0x0', 'totalBorrowBalanceUSD': '0x6f05b72089d1c31', 'totalFeesUSD': '0xd529ae9e86000', 'currentLtv': '0x0', 'currentLiquidationThreshold': '0x0', 'healthFactor': '0x0', 'healthFactorBelowThreshold': '0x1'}


In [81]:
params = {'_user': 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce' ,"_reserve":"cx072d8f95877a53df350e3dc3d8dba2f379037d42","_amount":1000000000000000000}

_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(LENDING_POOL_DATA_PROVIDER)\
    .method("balanceDecreaseAllowed")\
    .params(params)\
    .build()
response = icon_service.call(_call)
print(response)

0x43a0ad274cb19f39504a


In [86]:
params = {"_user":"hx91bf040426f226b3bfcd2f0b5967bbb0320525ce"}
# params={"_base":"USDb","_quote":"USD"}
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(LendingPoolDataProvider)\
    .method("getUserAccountData")\
    .params(params)\
    .build()
response = icon_service.call(_call)
print(response)


{'totalLiquidityBalanceUSD': '0x183b5f03f8fff821e3', 'totalCollateralBalanceUSD': '0x183b5f03f8fff821e3', 'totalBorrowBalanceUSD': '0x6f05b73f9c74aad', 'totalFeesUSD': '0xd529ae9e86000', 'currentLtv': '0x340aad21b3b700000', 'currentLiquidationThreshold': '0x717f', 'healthFactor': '0x36985798df8377294589', 'healthFactorBelowThreshold': '0x0'}


In [63]:
params ={"_reserveAddress":"cx072d8f95877a53df350e3dc3d8dba2f379037d42","_userAddress": "hx91bf040426f226b3bfcd2f0b5967bbb0320525ce","_useAsCollateral": 1}
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(LENDING_POOL_CORE) \
    .nid(NID) \
    .nonce(100) \
    .method("updateUserReserveUseAsCollateral")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

{'version': '0x3', 'from': 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce', 'to': 'cx162fe5b5d3276e9493de1fd440511f0844ca9a7c', 'stepLimit': '0x40e98', 'timestamp': '0x5b35e68b690d2', 'nid': '0x3', 'nonce': '0x64', 'dataType': 'call', 'data': {'method': 'updateUserReserveUseAsCollateral', 'params': {'_reserveAddress': 'cx072d8f95877a53df350e3dc3d8dba2f379037d42', '_userAddress': 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce', '_useAsCollateral': '0x1'}}, 'signature': 'KL/M3PTyJ6j3s9GFLtkZQkHSkRJiAuhfJ56/JI4wImldTmTwgX9PKeevGwajGq6ClvK99iM+dha6Potw3JK/9QE='}


{'txHash': '0x53e3c9d6fad192bf400c3a9b6f52fae48845e16c05f2cd68d9e54f129c08f25d',
 'blockHeight': 278419,
 'blockHash': '0x4ad50deba15abb533f771053471ba0a610762e298de144f7304a5d7d74b27394',
 'txIndex': 1,
 'to': 'cx162fe5b5d3276e9493de1fd440511f0844ca9a7c',
 'stepUsed': 165880,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 165880,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x

In [12]:
deployer_wallet.get_private_key()

'a691ef7d5601f9b5be4f9b9d80215159ea6ff0b88003e3d34e078d40e778b39a'